In [6]:
import pandas as pd

from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader

batch_size = 1

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")

json_path ="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/"

modelDimRed = SSDimRedModel(eval_mode=True).cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175, eval_mode=True).cuda()



In [2]:
modelConv.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_conv/emo_reco_best_ep216_acc_62"))
modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_dimred/emo_reco_500.pth"))

<All keys matched successfully>

In [4]:
import numpy as np
from numpy import ndarray


def generateParameters(device, model, dataset, labels):
    loader = DataLoader(dataset, shuffle=False, batch_size=1)
    d ={}

    for label in labels:
        d.update({label: []})

    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            dims, pred = model(X, return_with_dims= True)
            label = data_set.encoded_dataset.label_list[z[0].nonzero().cpu().numpy()[0][0]]
            dims = dims.cpu().numpy()[0]
            d[label].append(dims)
            if batch % 500 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")
    return d


def genAvgFull(fulldata: list[dict], label_list, modelNameList):
    fullDict = {}
    for label in label_list:
        fullDict.update({label: genAvgAndVar([fd[label] for fd in fulldata], modelNameList)})
    return fullDict

def genAvgAndVar(data: list[list[ndarray[float]]], modelNameList):
    d = {}
    for i in range(len(modelNameList)):
        mean = np.mean(np.asarray(data[i]), axis=0)
        var = np.var(np.asarray(data[i]), axis=0)
        d.update({modelNameList[i] : pd.Series([mean, var], index=["mean", "var"])})
    return pd.DataFrame(d)


def generateJson(data):
    str = "{"
    for i in range(len(data)):
        str += f"\"x{i+1}\": {data[i]}"
        str += "," if i < len(data)-1 else ""
    str += "}"
    return(str)

def writeJson(emotion, string, apendix=""):
    print(f"writing: {string} \n to {json_path+emotion}.json")
    file = open(json_path+f"{emotion}{apendix}.json", "w")
    file.write(string)
    file.close()

In [11]:
emo = generateParameters("cuda", modelDimRed, data_set, data_set.encoded_dataset.label_list)
conv_emo = generateParameters("cuda", modelConv, data_set, data_set.encoded_dataset.label_list)


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092
0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092


In [10]:
fullDict = genAvgFull([emo, conv_emo], data_set.encoded_dataset.label_list, ["red", "conv"])


In [14]:
print(f"neutral :  redDim: {fullDict['neutral']['red']['mean']}, conv: {fullDict['neutral']['conv']['mean']}")
print(f"happy   :  redDim: {fullDict['happy']['red']['mean']}, conv: {fullDict['happy']['conv']['mean']}")
print(f"sad     :  redDim: {fullDict['sad']['red']['mean']}, conv: {fullDict['sad']['conv']['mean']}")
print(f"angry   :  redDim: {fullDict['angry']['red']['mean']}, conv: {fullDict['angry']['conv']['mean']}")
print(f"fear    :  redDim: {fullDict['fear']['red']['mean']}, conv: {fullDict['fear']['conv']['mean']}")
print(f"disgust :  redDim: {fullDict['disgust']['red']['mean']}, conv: {fullDict['disgust']['conv']['mean']}")
print(f"surprise:  redDim: {fullDict['surprise']['red']['mean']}, conv: {fullDict['surprise']['conv']['mean']}")



neutral :  redDim: [ 0.6616187   0.73795056 -0.744872   -0.633462  ], conv: [-0.40242827 -0.9487567   0.21536633 -0.91296506]
happy   :  redDim: [-0.60835606 -0.6879876  -0.16170374 -0.5159379 ], conv: [-0.9162042   0.81222427 -0.94405884  0.55082613]
sad     :  redDim: [ 0.7676968   0.53488284  0.55854934 -0.69721246], conv: [-0.32716918  0.8042437   0.89900833 -0.5743747 ]
angry   :  redDim: [-0.6810923  -0.5163825   0.65164894  0.73342377], conv: [ 0.93623155 -0.94140714 -0.9219744   0.4013685 ]
fear    :  redDim: [0.69507694 0.6124118  0.42723086 0.6004365 ], conv: [ 0.91713196  0.89425254 -0.6344462   0.9102133 ]
disgust :  redDim: [ 0.6164171  -0.7668218   0.582811   -0.56092954], conv: [-0.79294395 -0.9409855   0.8996731   0.93437487]
surprise:  redDim: [ 0.68390673 -0.65616286  0.49200478  0.62585956], conv: [ 0.89045614  0.79051965 -0.9374066  -0.9384939 ]


In [22]:
writeJson("neutral", generateJson(fullDict['neutral']['conv']['mean']))
writeJson("happy", generateJson(fullDict['happy']['conv']['mean']))
writeJson("sad", generateJson(fullDict['sad']['conv']['mean']))
writeJson("angry", generateJson(fullDict['angry']['conv']['mean']))
writeJson("fear", generateJson(fullDict['fear']['conv']['mean']))
writeJson("disgust", generateJson(fullDict['disgust']['conv']['mean']))
writeJson("surprise", generateJson(fullDict['surprise']['conv']['mean']))

writing: {"x1": -0.4024282693862915,"x2": -0.9487566947937012,"x3": 0.21536633372306824,"x4": -0.9129650592803955} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/neutral.json
writing: {"x1": -0.9162042140960693,"x2": 0.812224268913269,"x3": -0.9440588355064392,"x4": 0.5508261322975159} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/happy.json
writing: {"x1": -0.32716917991638184,"x2": 0.8042436838150024,"x3": 0.8990083336830139,"x4": -0.5743746757507324} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/sad.json
writing: {"x1": 0.9362315535545349,"x2": -0.9414071440696716,"x3": -0.9219744205474854,"x4": 0.40136849880218506} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/angry.json
writing: {"x1": 0.9171319603919983,"x2": 0.8942525386810303,"x3": -0.6344462037086487,"x4": 0.91021329164505} 
 to /h

In [45]:
f"{modelConv.__class__}".split('.')[-1].split("'>")[0]


SSConvModel3Sec


# Umap Based

In [2]:
import umap
import pandas as pd
import numpy as np
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
from sklearn.decomposition import PCA

enc_ds= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/encoder_datasets/allEncodings_clip_like.pkl", device="cuda")

lst = [tens.numpy() for tens in enc_ds.encoded_dataset.encodedData["encoded"]]

df = enc_ds.encoded_dataset.encodedData

pca = umap.UMAP(n_components=4, n_neighbors=5000)
out = pca.fit(np.asarray(lst))
pcaOut = np.asarray([i for i in out.embedding_])
pcaOut = 2 * (pcaOut - pcaOut.min(axis=0)) / (pcaOut.max(axis=0) - pcaOut.min(axis=0)) - 1
#df["pca"]=[i for i in out.embedding_]
df["pca"] = pcaOut.tolist()
# pca = PCA(n_components=4)
# out = pca.fit(np.asarray(lst).T)
#
# # np.asarray(lst).T
# df["pca"]=[i for i in out.components_.T]

array = np.array(df.loc[df["clear_emotion"].isin(["happy"])]["pca"].values.tolist(), dtype=object)
emo_dict_mean =  {}

# for label in enc_ds.encoded_dataset.label_list:
#     pca = np.array(df.loc[df["clear_emotion"].isin([label])]["pca"].values.tolist(), dtype=object)
#     normalized = 2 * (pca - pca.min(axis=0)) / (pca.max(axis=0) - pca.min(axis=0)) - 1
#     emo_dict_mean.update({label: normalized.mean(axis=0)})
#     writeJson(label, generateJson(normalized.mean(axis=0)), apendix="pca")

In [4]:
len(df)

6092

In [ ]:
for label in enc_ds.encoded_dataset.label_list:
    pca = np.array(df.loc[df["clear_emotion"].isin([label])]["pca"].values.tolist(), dtype=object)
    #normalized = 2 * (pca - pca.min(axis=0)) / (pca.max(axis=0) - pca.min(axis=0)) - 1
    normalized = pca
    emo_dict_mean.update({label: normalized.mean(axis=0)})
    writeJson(label, generateJson(normalized.mean(axis=0)), apendix="pca")